In [2]:
from glob import glob
import pandas as pd
paths = glob('./data/*/*/*')
filedata = pd.DataFrame([path.split('/')[2:] for path in paths], columns=['subject', 'pose', 'filename'])
filedata['path'] = paths
filedata = filedata.sort_values(['subject', 'pose']).reset_index(drop=True)
filedata
data = []
for index, value in filedata.iterrows():
    df = pd.read_pickle(value['path'])
    df = df.rename_axis('time').reset_index()
    df['id'] = index
    cols = df.columns.to_list()
    df = df[cols[-1:] + cols[:-1]]
    data.append(df)
data = pd.concat(data)
data = data.reset_index(drop=True)
data

Category  id time force             ... FSR_for_coord                              
Position              A    B     C  ...             B         C         D    E    F
0          0    0   0.0  0.0   0.0  ...           0.0  0.000000  0.000000  0.0  0.0
1          0    1   0.0  0.0   0.0  ...           0.0  0.000000  0.952381  0.0  0.0
2          0    2   0.0  0.0   0.0  ...           0.0  0.000000  1.158730  0.0  0.0
3          0    3   0.0  0.0   0.0  ...           0.0  0.000000  1.333333  0.0  0.0
4          0    4   0.0  0.0   0.0  ...           0.0  0.000000  1.587302  0.0  0.0
...       ..  ...   ...  ...   ...  ...           ...       ...       ...  ...  ...
52144     72  635   0.0  0.0   0.0  ...           0.0  0.587302  2.857143  0.0  0.0
52145     72  636   0.0  0.0   0.0  ...           0.0  0.603175  2.730159  0.0  0.0
52146     72  637   0.0  0.0   0.0  ...           0.0  0.603175  2.619048  0.0  0.0
52147     72  638   0.0  0.0   0.0  ...           0.0  0.619048  2.539683  0.0  0.0
52148     72  639   0.0  0.0  10.0  ...           0.0  0.619048  2.476190  0.0  0.0

[52149 rows x 32 columns]

In [3]:
import torch

class FSRDataset(torch.utils.data.Dataset):
    def __init__(self, X_df, y_df, index):
        assert(len(X_df) == len(y_df))
        self.X_df = X_df
        self.y_df = y_df
        self.index = index

    def __len__(self):
        return len(self.index)
    
    def __getitem__(self, idx):
        import numpy as np
        X = self.X_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        y = self.y_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        return X, y

In [4]:
def get_index_splited_by_time(data, test_size=None):
    from sklearn.model_selection import train_test_split
    train_indexes = []
    test_indexes = []
    for _, group in data.groupby('id'):
        train_index, test_index = train_test_split(group.index, test_size=0.2, shuffle=False)
        train_indexes.append(train_index)
        test_indexes.append(test_index)
    return train_indexes, test_indexes

In [5]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        self.encoder = torch.nn.LSTM(input_size, hidden_size, num_layer)
        self.decoder = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
import torch
from ray.air import session
from sklearn.metrics import mean_absolute_error, mean_squared_error
import ray.train.torch
import ray.tune
import ray.air
import numpy as np
import model as net
import torch.utils.data
from ray.tune.schedulers import ASHAScheduler

def train_loop_per_worker(config, data):
    # ray.train.torch.enable_reproducibility()
    model_name = config['model_name']
    model_args = config['model_args']
    criterion_name = config['criterion_name']
    optimizer_name = config['optimizer_name']
    lr = config['lr']

    model = getattr(net, model_name)(**model_args)
    # model = ray.train.torch.prepare_model(model)
    criterion = getattr(torch.nn, criterion_name)()
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    train_index, test_index = get_index_splited_by_time(data)
    train_dataset = FSRDataset(data['FSR_for_force'], data['force'], train_index)
    test_dataset = FSRDataset(data['FSR_for_force'], data['force'], test_index)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)
    # train_loader = ray.train.torch.prepare_data_loader(train_loader)
    # test_loader = ray.train.torch.prepare_data_loader(test_loader)
    
    while True:
        model.train()
        criterion.train()
        for X, y in train_loader:
            pred = model(X)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        criterion.eval()
        with torch.no_grad():
            mae = []
            mse = []
            num = []
            for X, y in test_loader:
                pred = model(X)
                mae.append(mean_absolute_error(y, pred.cpu().detach()))
                mse.append(mean_squared_error(y, pred.cpu().detach()))
                num.append(len(y))
            mae = np.array(mae)
            mse = np.array(mse)
            num = np.array(num)
            mae = (mae * num).sum() / sum(num)
            mse = (mse * num).sum() / sum(num)
            
            rmse = mse ** 0.5
            checkpoint = ray.train.torch.TorchCheckpoint.from_model(model)
            session.report({'metric/mae': mae, 'metric/rmse': rmse}, checkpoint=checkpoint)

scheduler = ASHAScheduler(
    max_t=100,
    grace_period=1,
    reduction_factor=2,
    brackets=1,
    
)
tuner = ray.tune.Tuner(
    trainable=ray.tune.with_parameters(train_loop_per_worker, data=data),
    tune_config=ray.tune.TuneConfig(
        metric='metric/rmse',
        mode='min',
        scheduler=scheduler,
        num_samples=20,
    ),
    param_space={
        'lr':ray.tune.loguniform(1e-5, 1e-2),
        'model_name':'LSTM',
        'model_args':{
            'input_size':6, 
            'hidden_size':ray.tune.choice([8, 16, 32, 64, 128, 256, 512, 1024]), 
            'num_layer':ray.tune.randint(1, 8), 
            'output_size':6,
        },
        'criterion_name': 'MSELoss',
        'optimizer_name': 'Adam',
    },
    run_config=ray.air.RunConfig(
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=2,
            checkpoint_score_attribute='metric/rmse',
            checkpoint_score_order='min',
        ),
    ),
)
results = tuner.fit()

2023-06-28 11:56:27,423	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-06-28 11:56:28,632	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


Trial name,date,done,hostname,iterations_since_restore,metric/mae,metric/rmse,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_loop_per_worker_60e19_00000,2023-06-28_11-57-00,True,DESKTOP-0P789CI,2,236.167,552.147,172.26.215.93,102152,True,22.2064,10.6081,22.2064,1687921020,2,60e19_00000
train_loop_per_worker_60e19_00001,2023-06-28_11-56-43,True,DESKTOP-0P789CI,1,236.477,553.426,172.26.215.93,102153,True,6.4786,6.4786,6.4786,1687921003,1,60e19_00001
train_loop_per_worker_60e19_00002,2023-06-28_12-05-47,True,DESKTOP-0P789CI,100,198.565,445.429,172.26.215.93,102154,True,550.424,5.26463,550.424,1687921547,100,60e19_00002
train_loop_per_worker_60e19_00003,2023-06-28_12-00-05,True,DESKTOP-0P789CI,100,233.539,539.369,172.26.215.93,102155,True,208.28,1.91172,208.28,1687921205,100,60e19_00003
train_loop_per_worker_60e19_00004,2023-06-28_11-59-27,True,DESKTOP-0P789CI,100,236.314,553.051,172.26.215.93,102156,True,170.259,3.27323,170.259,1687921167,100,60e19_00004
train_loop_per_worker_60e19_00005,2023-06-28_13-01-16,True,DESKTOP-0P789CI,100,92.3027,177.019,172.26.215.93,102157,True,3879.27,23.9743,3879.27,1687924876,100,60e19_00005
train_loop_per_worker_60e19_00006,2023-06-28_12-04-14,True,DESKTOP-0P789CI,100,127.672,253.093,172.26.215.93,102158,True,457.529,4.73469,457.529,1687921454,100,60e19_00006
train_loop_per_worker_60e19_00007,2023-06-28_11-56-46,True,DESKTOP-0P789CI,1,236.475,553.435,172.26.215.93,102159,True,9.6204,9.6204,9.6204,1687921006,1,60e19_00007
train_loop_per_worker_60e19_00008,2023-06-28_12-51-57,True,DESKTOP-0P789CI,4,235.796,543.124,172.26.215.93,102153,True,3313.49,629.228,3313.49,1687924317,4,60e19_00008
train_loop_per_worker_60e19_00009,2023-06-28_11-56-55,True,DESKTOP-0P789CI,2,236.226,551.415,172.26.215.93,102159,True,8.40573,4.11546,8.40573,1687921015,2,60e19_00009


2023-06-28 13:01:16,676	INFO tune.py:1111 -- Total run time: 3888.04 seconds (3887.98 seconds for the tuning loop).
